
## NTRU-Encrypt

## Parâmetros
Vamos usar
+ $d \ge 6$ --
  parâmetro de segurança
+ $N$ --
  um primo maior ou igual  2^d
  
+ $p$ --
  um pequeno primo  3,  5 ou  7
 
+ $q$ --
  um primo da ordem de grandeza de  $p*N$
 

 

In [50]:
import hashlib

d = 10

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

10 3109 3 1031


### anéis de polinómios

In [5]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

### geração aleatória, arredondamento módulo $q$ e compração módulo $p$

In [11]:
def vec(n):
    return  [choice([-1,0,1]) for k in range(n)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

# comparação módulo 'p'
#def equalp(x,y,p):
#    return  all([(a-b)%p == 0  for (a,b) in zip(x,y)] )

def hashmsg(m,h):
    m = ''.join([str(x) for x in m])
    rseed = hashlib.sha512(hm)
    return rseed

### classe NTRU

In [35]:
class NTRU(object):
    
    #KeyGen
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec(N)); f = 1 + p*F
        # gerar as chaves
        G = Q(vec(N)) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
        
    
    def padding(self,m):
        if(len(m) >= (N-173)):
            print("Message is too short ")
        elif(len(m) <= 0):
            print("Message length is invalid")
        else:
            bit = bin(hello)
            print(bit)
        
        
    def encrypt(self,m):
        m = padding(m)
        r = Q(vec(N))
        t = r*self
        mask = Q(vec())
        #mess = (pad - mask) mod p
        #c = t + mess
        
        return r*self.h + Q(m)
        #return c

    def decrypt(self,e):
        a = e*self.f
        return prnd(qrnd(a))


In [49]:
def padding(msg):
        print(msg)
        msg = ''.join("{0:08b}".format(x) for x in msg)
        print(msg)
        messlen = len(msg)/8
        print(messlen)
        
        if messlen >= (N-173):
            print("Message is too large")
            return;
        if messlen <= 0:
            print("Message length is invalid")
            return;
            
        r = N - (167 + 6 + messlen*8)
        lr = list(0 for i in (0..r-1))
    
        messlen = "{0:06b}".format(int(messlen))
        print(messlen)
        messlen = [int(d) for d in messlen[:6]];
        print(messlen)
        
        msg = msg.replace('-', '1')
        msg = list(msg)
        
        m = msg + lr + vec(167) + messlen
    
        return m
    
m = vec(4)
cena = padding(m)

[0, 1, -1, -1]
0000000000000001-0000001-0000001
4
000100
[0, 0, 0, 1, 0, 0]


### Teste

In [3]:
# Uma instância NTRU
K = NTRU() 
# Uma mensagem aleatória
m = vec()
print(m)
# Cifrar
e = K.encrypt(m)  
# Decifrar e Verificar
m == K.decrypt(e)


NameError: global name 'Q' is not defined

## Reticulado $L(h)$

In [ ]:
# Construção da matriz geradora por blocos

import sage.modules.free_module_integer as fmi
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html

class Lat(NTRU):
    def __init__(self):
        super(Lat,self).__init__()
        B1 = identity_matrix(ZZ,N); Bq = q*B1; B0 = matrix(ZZ,N,N,[0]*(N^2))
        h = qrnd(self.h)
        # rodar um vetor
        H = [h]
        for k in range(N-1):
            h = [h[-1]] + h[:-1]   # shift right rotate
            H = H + [h]
        H = matrix(ZZ,N,N,H)
        self.G = block_matrix([[Bq,B0],[H,B1]])      

In [ ]:
Lh = fmi.IntegerLattice(Lat().G)